### **Optimizing Role-Based Access Control (RBAC) in FastAPI**
Since you are structuring your FastAPI project for scalability, let’s make role-based access **cleaner, reusable, and maintainable**.

---

### **🔹 Step 1: Create a Reusable `role_required` Dependency**
Instead of repeating `Depends(role_required(...))` in every route, let’s create a reusable dependency.

#### ✅ **Improved `role_required` Function**
```python
from fastapi import Depends, HTTPException, status
from typing import List
from some_auth_module import get_current_user  # Your function to fetch the current user

def role_required(allowed_roles: List[str]):
    async def check_user_role(current_user: dict = Depends(get_current_user)):
        if current_user.get("role") not in allowed_roles:
            raise HTTPException(
                status_code=status.HTTP_403_FORBIDDEN,
                detail="You do not have permission to access this resource.",
            )
        return current_user  # Allows access if role matches

    return check_user_role
```
---
### **🔹 Step 2: Apply It Efficiently in Routes**
Now, we use **two optimized approaches** based on whether we need user details inside the function.

#### ✅ **For Routes That Need `current_user` (Use `Depends` in Parameters)**
If you need user details inside the function, use this approach:
```python
@authorization_route.get("/")
async def get_authorization(current_user: dict = Depends(role_required(["user", "admin"]))):
    return {"message": "Hello, User!", "user": current_user}
```
**Why?**  
✔ You **get access** to `current_user` inside the function.  
✔ Useful if you need to customize the response.

---

#### ✅ **For Routes That Only Require Access Control (Use `dependencies=[]`)**
If you just want to restrict access without needing `current_user` in the function:
```python
@authorization_route.get("/admin-only", dependencies=[Depends(role_required(["admin"]))])
async def admin_dashboard():
    return {"message": "Welcome, Admin!"}
```
**Why?**  
✔ Cleaner function signature.  
✔ Ensures role-based access without modifying function logic.

---

### **🔹 Step 3: Apply Global Role-Based Access Control on a Router**
Instead of adding `Depends(role_required(...))` in every route, **apply it at the router level** to protect an entire section.

#### ✅ **Protecting an Entire Router (e.g., Admin Routes)**
```python
from fastapi import APIRouter, Depends

admin_router = APIRouter(
    prefix="/admin",
    tags=["Admin"],
    dependencies=[Depends(role_required(["admin"]))],  # Protects all routes inside this router
)

@admin_router.get("/dashboard")
async def admin_dashboard():
    return {"message": "Welcome to the Admin Dashboard!"}

@admin_router.get("/stats")
async def admin_stats():
    return {"message": "Admin stats data"}
```
**Now, all routes inside `admin_router` require an "admin" role!** 🚀  
✔ Saves repetitive checks.  
✔ Enforces access control **for all admin routes**.

---

### **Final Thoughts**
1️⃣ **Use `Depends(role_required(...))` inside function parameters** if you need `current_user`.  
2️⃣ **Use `dependencies=[Depends(...)]`** when you just need access control without using `current_user`.  
3️⃣ **Apply dependencies at the router level** for cleaner and scalable role-based access across multiple routes.  


# Customized onld methods as per new role_required codebase.


```python
from fastapi import APIRouter, Depends
from src.domains.authorization.utils import role_required


authorization_route = APIRouter()

# 1️⃣ Use Depends(role_required(...)) inside function parameters if you need current_user.
# 2️⃣ Use dependencies=[Depends(...)] when you just need access control without using current_user.
# 3️⃣ Apply dependencies at the router level for cleaner and scalable role-based access across multiple routes.


#  1st way : If you need user details inside the function, use this approach:

@authorization_route.get('/user-only')
async def get_authorization1(current_user: dict = Depends(role_required(["user"]))):
    return current_user


@authorization_route.get("/user-admin")
async def get_authorization2(current_user: dict = Depends(role_required(["user", "admin"]))):
    return {"message": "Hello, User!", "user": current_user}


#  2nd way : For Routes That Only Require Access Control (Use dependencies=[])
@authorization_route.get("/admin-only", dependencies=[Depends(role_required(["admin"]))])
async def admin_dashboard():
    return {"message": "Welcome, Admin!"}


@authorization_route.get("/user-dashboard", dependencies=[Depends(role_required(["user"]))])
async def user_dashboard():
    return {"message": "Welcome, User!"}



# 3rd way : Apply Global Role-Based Access Control on a Router

admin_router = APIRouter(
    prefix="/admin",
    tags=["Admin"],
    dependencies=[Depends(role_required(["admin"]))],  # Protects all routes inside this router
)

@admin_router.get("/dashboard")
async def admin_dashboard():
    return {"message": "Welcome to the Admin Dashboard!"}

@admin_router.get("/stats")
async def admin_stats():
    return {"message": "Admin stats data"}




```

## New Role Required Method:

``` python

def role_required(allowed_roles: List[str]):
    """Dependency to check if the user has the required role."""
    async def dependency(current_user: dict = Depends(get_current_user)):
        user_role = current_user.get("role")  # Ensure role is extracted correctly
        if user_role not in allowed_roles:
            raise HTTPException(
                status_code=status.HTTP_403_FORBIDDEN,
                detail="You do not have permission to access this resource.",
            )
        return current_user  # Allows access if role matches

    return dependency

```